In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict, NamedSplit

In [ ]:
dataset_path = "./data/used_car_sales/car_prices.csv"

In [ ]:
df = pd.read_csv(dataset_path)

## EDA Dataset

In [ ]:
df.columns

Index(['year', 'make', 'model', 'trim', 'body', 'transmission', 'vin', 'state',
       'condition', 'odometer', 'color', 'interior', 'seller', 'mmr',
       'sellingprice', 'saledate'],
      dtype='object')

In [ ]:
df.dtypes

year              int64
make             object
model            object
trim             object
body             object
transmission     object
vin              object
state            object
condition       float64
odometer        float64
color            object
interior         object
seller           object
mmr             float64
sellingprice    float64
saledate         object
dtype: object

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558837 entries, 0 to 558836
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          558837 non-null  int64  
 1   make          548536 non-null  object 
 2   model         548438 non-null  object 
 3   trim          548186 non-null  object 
 4   body          545642 non-null  object 
 5   transmission  493485 non-null  object 
 6   vin           558833 non-null  object 
 7   state         558837 non-null  object 
 8   condition     547017 non-null  float64
 9   odometer      558743 non-null  float64
 10  color         558088 non-null  object 
 11  interior      558088 non-null  object 
 12  seller        558837 non-null  object 
 13  mmr           558799 non-null  float64
 14  sellingprice  558825 non-null  float64
 15  saledate      558825 non-null  object 
dtypes: float64(4), int64(1), object(11)
memory usage: 68.2+ MB


In [ ]:
def pre_process_data(df):
    df = df[~df.sellingprice.isna()].reset_index(drop=True)
    df["saledate"] = pd.to_datetime(df["saledate"], errors="coerce", utc=True)
    df = df[df["saledate"].between("1940-01-01", "2025-01-01")].reset_index(drop=True)
    df["sales_year"] = df.saledate.apply(lambda x: x.year)
    df = df[~df.odometer.isna()].reset_index(drop=True)

    columns_to_use = [
        "year",
        "make",
        "model",
        "trim",
        "body",
        "transmission",
        "state",
        "condition",
        "odometer",
        "color",
        "interior",
        "mmr",
        "sales_year",
        "sellingprice",
    ]

    df = df[columns_to_use]
    df["condition"] = df.condition.fillna("not given").astype(str)
    df = df.fillna("not given")

    return df

In [ ]:
df = pre_process_data(df)

/tmp/ipykernel_12849/1938723959.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["saledate"] = pd.to_datetime(df["saledate"], errors="coerce", utc=True)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558705 entries, 0 to 558704
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          558705 non-null  int64  
 1   make          558705 non-null  object 
 2   model         558705 non-null  object 
 3   trim          558705 non-null  object 
 4   body          558705 non-null  object 
 5   transmission  558705 non-null  object 
 6   state         558705 non-null  object 
 7   condition     558705 non-null  object 
 8   odometer      558705 non-null  float64
 9   color         558705 non-null  object 
 10  interior      558705 non-null  object 
 11  mmr           558705 non-null  float64
 12  sales_year    558705 non-null  int64  
 13  sellingprice  558705 non-null  float64
dtypes: float64(3), int64(2), object(9)
memory usage: 59.7+ MB


In [ ]:
def prepare_input_text(row):
    instruct_txt = "Properties of car: "
    for clm in [
        "year",
        "make",
        "model",
        "trim",
        "body",
        "transmission",
        "state",
        "condition",
        "odometer",
        "color",
        "interior",
        "mmr",
        "sales_year",
    ]:
        instruct_txt += f"{clm} is {row[clm]}, "

    return instruct_txt.strip()[:-1]+'.'

In [ ]:
df_new = pd.DataFrame(columns = ['input','output'])

In [ ]:
df_new

,input,output


In [ ]:
df_new['input'] = df.apply(lambda x: prepare_input_text(x),axis=1)

In [ ]:
df_new['output'] = df.sellingprice

In [ ]:
df_new.iloc[0].input

'Properties of car: year is 2015, make is Kia, model is Sorento, trim is LX, body is SUV, transmission is automatic, state is ca, condition is 5.0, odometer is 16639.0, color is white, interior is black, mmr is 20500.0, sales_year is 2014.'

In [ ]:
df_new

,input,output
0,"Properties of car: year is 2015, make is Kia, ...",21500.0
1,"Properties of car: year is 2015, make is Kia, ...",21500.0
2,"Properties of car: year is 2014, make is BMW, ...",30000.0
3,"Properties of car: year is 2015, make is Volvo...",27750.0
4,"Properties of car: year is 2014, make is BMW, ...",67000.0
...,...,...
558700,"Properties of car: year is 2015, make is Kia, ...",33000.0
558701,"Properties of car: year is 2012, make is Ram, ...",30800.0
558702,"Properties of car: year is 2012, make is BMW, ...",34000.0
558703,"Properties of car: year is 2015, make is Nissa...",11100.0


In [ ]:
df_new.input.apply(lambda x: len(x)).describe()

count    558705.000000
mean        245.766997
std           6.841599
min         226.000000
25%         241.000000
50%         244.000000
75%         249.000000
max         287.000000
Name: input, dtype: float64

In [ ]:
dataset = Dataset.from_pandas(df_new, split=NamedSplit("train"))

In [ ]:
train_testvalid = dataset.train_test_split(test_size=0.2)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid["test"].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
dataset_new = DatasetDict(
    {
        "train": train_testvalid["train"],
        "test": test_valid["test"],
        "valid": test_valid["train"],
    }
)

In [ ]:
dataset_new

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 446964
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 55871
    })
    valid: Dataset({
        features: ['input', 'output'],
        num_rows: 55870
    })
})

In [ ]:
dataset_new.save_to_disk("./data/used_car_sales/prepared_car_sales_data")

Saving the dataset (0/1 shards):   0%|          | 0/446964 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/55871 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/55870 [00:00<?, ? examples/s]